## Functions are non-data descriptors!

## Comments:

- If not using instance and set_name: 
    - I should make a weakref when setting and remove the key dictionary during deletion
    - 

In [232]:
class MyDescriptor():
    
    def __init__(self):
        print('MyDescriptor __init__ ')
        #self is an instance of MyDescriptor()
        print('-'*30)
    
    def __set_name__(self, owner_class, prop_name):
        print('calling __set_name__ ...')
        print(f'owner_class={owner_class}')
        # get a hook on the property name. It solves the problem
        # x = MyDescriptor('x') , i.e. having to specify 'x' during creation
        self.prop_name = prop_name
        print('-'*30)
    
    def __get__(self, instance, owner_class):
        print('calling __get__ ...')
        if instance is None:#called from class MyClass
            return self #return instance of descriptor
        return instance.__dict__.get(self.prop_name, None)
        print('-'*30)
    
    def __set__(self, instance, value):
        print('calling __set__ ...')
        # by setting the prop_name it doesn't assume 
        instance.__dict__[self.prop_name] = value
        print('-'*30)
    

In [233]:
class MyClass:
    
    x = MyDescriptor()
    y = MyDescriptor()
    
    def __init__(self):
        print('calling MyClass __init__ ...')
        #self is an instance of my class
        pass
    
    

MyDescriptor __init__ 
------------------------------
MyDescriptor __init__ 
------------------------------
calling __set_name__ ...
owner_class=<class '__main__.MyClass'>
------------------------------
calling __set_name__ ...
owner_class=<class '__main__.MyClass'>
------------------------------


In [234]:
MyClass.__dict__

mappingproxy({'__module__': '__main__',
              'x': <__main__.MyDescriptor at 0x1b6ae27d588>,
              'y': <__main__.MyDescriptor at 0x1b6ae27d548>,
              '__init__': <function __main__.MyClass.__init__(self)>,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

### get the descriptor back when called from class itself

In [235]:
MyClass.x

calling __get__ ...


In [236]:
MyClass.y

calling __get__ ...


In [237]:
inst = MyClass()
inst.__dict__

calling MyClass __init__ ...


{}

In [238]:
inst.x

calling __get__ ...


In [240]:
inst.x = 5
inst.x

calling __set__ ...
------------------------------
calling __get__ ...


5

In [242]:
inst.y

calling __get__ ...


In [247]:
MyClass.x.__get__(inst, MyClass)

calling __get__ ...
calling __get__ ...


5

In [248]:
descr = MyClass.x  #get a hook on the descriptor
# calling get due to .dot notation

calling __get__ ...


In [249]:
descr.__get__(inst, MyClass)  # calling 

calling __get__ ...


5

## DesctiptorClass.__get__(DescriptorInstance, None means called from class, owner_type Myclass)

In [257]:
MyDescriptor.__get__(MyClass.__dict__['x'], None, MyClass)

calling __get__ ...


In [258]:
inst = MyClass()

calling MyClass __init__ ...


## Calling descriptor from instance (bound method)

In [259]:
MyDescriptor.__get__(MyClass.__dict__['x'], inst, MyClass)

calling __get__ ...


In [260]:
# same as
MyClass.__dict__['x'].__get__(inst, MyClass)

# MyClass.__dict__['x'] will be passed into the 'self'

calling __get__ ...


### Functions are non-data descriptors

In [226]:
def f():
    pass

type(f)

function

In [227]:
def add(a,b):
    return a+b

In [228]:
import sys
main_module = sys.modules['__main__']
main_module

<module '__main__'>

In [254]:
type(add)  # This is the 'class' of a function

function

## calling from the instance

In [253]:
type(add).__get__(add, None, main_module)

<function __main__.add(a, b)>

### it returned the descriptor

In [261]:
# same as calling as bound method

add.__get__(None, main_module)

<function __main__.add(a, b)>

## Recall that:

In [ ]:
# Called from instance --> returns attribute
MyDescriptor.__get__(MyClass.__dict__['x'], inst, MyClass)

# same as
MyClass.__dict__['x'].__get__(inst, MyClass)

# MyClass.__dict__['x'] will be passed into the 'self'

In [ ]:
# Called from class (None) --> returns 
MyDescriptor.__get__(MyClass.__dict__['x'], None, MyClass)

## Hence it is evident that functions are non-data Descriptors!

In [ ]:
class functionDescriptor():
    '''
    Only implement __get__ and NOT __set__
    '''
        
    def __init__(self, fn):
        print('functionDescriptor __init__ ')
        #self is an instance of MyDescriptor()
        self.fn = fn.copy()
        print('-'*30)
    
    def __set_name__(self, owner_class, prop_name):
        print('calling __set_name__ ...')
        print(f'owner_class={owner_class}')
        # get a hook on the property name. It solves the problem
        # x = MyDescriptor('x') , i.e. having to specify 'x' during creation
        self.prop_name = prop_name
        print('-'*30)
    
    def __get__(self, instance, owner_class):
        print('calling __get__ ...')
        if instance is None:#called from class MyClass
            return self #return instance of descriptor
        #return self.fn(self.prop_name)
        return instance.

In [275]:
class functionDescriptor():
    '''
    Only implement __get__ and NOT __set__
    '''
        
    def __init__(self, fn):
        print('functionDescriptor __init__ ')
        #self is an instance of MyDescriptor()
        self.fn = fn
        print('-'*30)
    
    def __set_name__(self, owner_class, prop_name):
        print('calling __set_name__ ...')
        print(f'owner_class={owner_class}')
        # get a hook on the property name. It solves the problem
        # x = MyDescriptor('x') , i.e. having to specify 'x' during creation
        self.prop_name = prop_name
        print('-'*30)
    
    def __get__(self, instance, owner_class):
        print('calling __get__ ...')
        if instance is None:#called from class MyClass
            return self #return instance of descriptor
        return self.fn
        #return self.fn(self.prop_name)
        #return copy.deepcopy(self.fn)
        print('-'*30)

In [324]:
class Person:
    
    greet = functionDescriptor(lambda self : print(f'{self} greets you!'))

functionDescriptor __init__ 
------------------------------
calling __set_name__ ...
owner_class=<class '__main__.Person'>
------------------------------


In [277]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'greet': <__main__.functionDescriptor at 0x1b6ae299ec8>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [278]:
Person.greet

calling __get__ ...


In [279]:
p = Person()
p.__dict__

{}

In [280]:
p.greet  # without the parenthesis though...

calling __get__ ...


<function __main__.Person.<lambda>(self)>

In [281]:
f1 = p.greet
f2 = p.greet

calling __get__ ...
calling __get__ ...


In [282]:
f1 is f2

True

In [299]:
import copy

In [316]:
class functionDescriptor():
    '''
    Only implement __get__ and NOT __set__
    '''
    
    class CallableFunc:
        def __init__(self, fn, instance):
            self.fn = fn 
            self.instance = instance
            
        def __call__(self):
            return self.fn(self.instance) 
        
    def __init__(self, fn):
        print('functionDescriptor __init__ ')
        #self is an instance of MyDescriptor()
        #self.my_callable = callable_func(fn)
        self.fn = fn
        print('-'*30)
    
    def __set_name__(self, owner_class, prop_name):
        print('calling __set_name__ ...')
        print(f'owner_class={owner_class}')
        # get a hook on the property name. It solves the problem
        # x = MyDescriptor('x') , i.e. having to specify 'x' during creation
        self.prop_name = prop_name
        print('-'*30)
    
    def __get__(self, instance, owner_class):
        print('calling __get__ ...')
        if instance is None:#called from class MyClass
            return self #return instance of descriptor
        
        my_callable = functionDescriptor.CallableFunc(self.fn, instance)
        
         # return new method object everytime thats bound to instnace
        return my_callable
        
        
        #return self.fn
        #return self.fn(self.prop_name)
        #return copy.deepcopy(self.fn)
        # return something callable
        print('-'*30)

In [317]:
class Person:
    
    greet = functionDescriptor(lambda self : print(f'{self} greets you!'))

functionDescriptor __init__ 
------------------------------
calling __set_name__ ...
owner_class=<class '__main__.Person'>
------------------------------


In [318]:
p = Person()
p.__dict__

{}

In [319]:
p.greet  # without the parenthesis though...

calling __get__ ...


In [320]:
f1 = p.greet
f2 = p.greet

calling __get__ ...
calling __get__ ...


In [321]:
f1 is f2

False

In [322]:
f1()

<__main__.Person object at 0x000001B6AE26FF08> greets you!


In [323]:
f2()

<__main__.Person object at 0x000001B6AE26FF08> greets you!


In [2]:
class functionDescriptor():
    '''
    Only implement __get__ and NOT __set__
    '''
    
    class CallableFunc:
        def __init__(self, fn, instance):
            self.fn = fn 
            self.instance = instance
            
        def __call__(self):
            return self.fn(self.instance) 
        
    def __init__(self, fn):
        print('functionDescriptor __init__ ')
        self.fn = fn
        print('-'*30)
    
    def __set_name__(self, owner_class, prop_name):
        print('calling __set_name__ ...')
        print(f'owner_class={owner_class}')
        self.prop_name = prop_name
        print('-'*30)
    
    def __get__(self, instance, owner_class):
        print('calling __get__ .. .')
        if instance is None:#called from class MyClass
            return self #return instance of descriptor
        
        my_callable = functionDescriptor.CallableFunc(self.fn, instance)
        
        # return new method object everytime thats bound to instnace
        return my_callable
        print('-'*30)

In [3]:
class Person:
    
    greet = functionDescriptor(lambda self : print(f'{self} greets you!'))

functionDescriptor __init__ 
------------------------------
calling __set_name__ ...
owner_class=<class '__main__.Person'>
------------------------------


In [4]:
p = Person

In [5]:
p.greet

calling __get__ .. .
